In [1]:
from libraries import *
from logger import setup_logger
from CVAE_model import E3ScreenDataset, VAE, Encoder, Decoder, get_loss_fn, EmbeddingLayer
from Training import train_model_CVAE, train_model_multilab, evaluate_model_CVAE, evaluate_model_CVAE_multilab

import torch
from torch.utils.data import Dataset, DataLoader

import bisect
from itertools import accumulate
import argparse

import sys
import os
import logger
import torch.nn as nn
import torch
from torch import optim


In [2]:
n_inputs = 1041
n_cond_in= 6
n_cond= 64
n_latents= 64
save_dir= './Notebooks/CombinatorialPerturbations/outputs'
model_dir= 'model_alpha_0_second'
datafile= "./Notebooks/CombinatorialPerturbations/dataset/adataComboGroupsAll.h5ad"
batch_size= 1000
project_dir= "/home/eraslab1/Projects/E3Ligase/analysisSingle"


In [5]:
os.getcwd()
os.chdir(project_dir)

In [3]:
net = VAE(n_inputs=n_inputs,
          n_latents=n_latents,
          n_cond=n_cond,
          n_cond_in=n_cond_in)

In [6]:
net.load_state_dict(torch.load(os.path.join(save_dir,model_dir,"best.pth"))['state_dict'])

<All keys matched successfully>

In [7]:
dataset = E3ScreenDataset(datafile)
dataloader = DataLoader(dataset, 
                        batch_size=batch_size,
                        drop_last=False,
                        shuffle=False,
                        num_workers=50)

In [15]:
total_embeddings = []
allConditions = []
for sample in dataloader:
    with torch.no_grad():
        #input = sample['y'].cuda() if next(model.parameters()).is_cuda else sample['y']
        #condition = sample['X'].cuda() if next(model.parameters()).is_cuda else sample['X']
        input = sample['y']
        condition = sample['X']
        embeddings = net.inference(input, condition)
        
        total_embeddings.append(embeddings)
        allConditions.append(condition)


In [16]:
total_embeddings = torch.cat(total_embeddings, 0)
allConditions = torch.cat(allConditions, 0)

In [17]:
total_embeddings = total_embeddings.numpy()
allConditions = allConditions.numpy()

In [21]:
total_embeddings.shape

(97416, 64)

In [22]:
allConditions

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0.]], dtype=float32)